# Model Validation - Modular Implementation**Data Split**: 2020-2024 (train) / 2025 (test)  **Backtest**: 2022, 2023, 2024 independently  **Models**: Prophet baseline vs Prophet+XGBoost hybrid

In [ ]:
import syssys.path.append('.')from src import load_data, split_train_test, run_multiple_backtests, evaluate_predictions, print_metricsimport numpy as npimport warningswarnings.filterwarnings('ignore')print("Modules loaded successfully")

In [ ]:
# Load and preprocess datadata_path = 'Data_set/stock_features_clean.csv'sector_df = load_data(data_path)

In [ ]:
# Train/Test split (no data leakage)train_df, test_df = split_train_test(sector_df, train_end_year=2024, test_year=2025)sectors = sorted(train_df['Sector'].unique())print(f"Sectors: {sectors}")

In [ ]:
# Run backtests for 2022, 2023, 2024backtest_years = [2022, 2023, 2024]backtest_results = run_multiple_backtests(train_df, backtest_years, sectors, alpha=0.5)print(f"\nBacktests completed for {len(backtest_results)} years")

In [ ]:
# Evaluate each yearall_metrics = []for results in backtest_results:    actuals = np.array(list(results['actuals'].values()))    prophet_preds = np.array(list(results['prophet_preds'].values()))    hybrid_preds = np.array(list(results['hybrid_preds'].values()))        metrics = evaluate_predictions(actuals, prophet_preds, hybrid_preds)    print_metrics(metrics, results['year'])        all_metrics.append(metrics)

In [ ]:
# Average performance across backtest yearsavg_prophet_mape = np.mean([m['prophet_mape'] for m in all_metrics])avg_hybrid_mape = np.mean([m['hybrid_mape'] for m in all_metrics])avg_prophet_rmse = np.mean([m['prophet_rmse'] for m in all_metrics])avg_hybrid_rmse = np.mean([m['hybrid_rmse'] for m in all_metrics])avg_prophet_spearman = np.mean([m['prophet_spearman'] for m in all_metrics])avg_hybrid_spearman = np.mean([m['hybrid_spearman'] for m in all_metrics])print("\n" + "="*80)print(f"Average Performance ({backtest_years[0]}-{backtest_years[-1]})")print("="*80)print(f"{'Metric':<20s} | {'Prophet':>10s} | {'Hybrid':>10s} | {'Improve':>10s}")print("-"*80)print(f"{'MAPE (%)':<20s} | {avg_prophet_mape:>10.2f} | {avg_hybrid_mape:>10.2f} | {avg_prophet_mape-avg_hybrid_mape:>10.2f}")print(f"{'RMSE':<20s} | {avg_prophet_rmse:>10.4f} | {avg_hybrid_rmse:>10.4f} | {avg_prophet_rmse-avg_hybrid_rmse:>10.4f}")print(f"{'Spearman':<20s} | {avg_prophet_spearman:>10.4f} | {avg_hybrid_spearman:>10.4f} | {avg_hybrid_spearman-avg_prophet_spearman:>10.4f}")print("="*80)

In [ ]:
# 2025 Hold-out Testfrom src.backtest import run_backtestprint("\n" + "#"*80)print("# 2025 Hold-out Test")print("#"*80)results_2025 = run_backtest(train_df, test_df, 2025, sectors, alpha=0.5)actuals_2025 = np.array(list(results_2025['actuals'].values()))prophet_preds_2025 = np.array(list(results_2025['prophet_preds'].values()))hybrid_preds_2025 = np.array(list(results_2025['hybrid_preds'].values()))metrics_2025 = evaluate_predictions(actuals_2025, prophet_preds_2025, hybrid_preds_2025)print_metrics(metrics_2025, 2025)

In [ ]:
# Sector rankings comparisonprint("\n" + "="*80)print("2025 Sector Rankings")print("="*80)sorted_actual = sorted(results_2025['actuals'].items(), key=lambda x: x[1], reverse=True)sorted_prophet = sorted(results_2025['prophet_preds'].items(), key=lambda x: x[1], reverse=True)sorted_hybrid = sorted(results_2025['hybrid_preds'].items(), key=lambda x: x[1], reverse=True)print(f"{'Rank':<6s} | {'Actual':<30s} | {'Prophet':<30s} | {'Hybrid':<30s}")print("-"*100)for i in range(len(sorted_actual)):    actual_sector = sorted_actual[i][0]    prophet_sector = sorted_prophet[i][0] if i < len(sorted_prophet) else '-'    hybrid_sector = sorted_hybrid[i][0] if i < len(sorted_hybrid) else '-'        print(f"{i+1:<6d} | {actual_sector:<30s} | {prophet_sector:<30s} | {hybrid_sector:<30s}")print("="*80)

## Conclusion**Model Validation Complete**- Training: 2020-2024 (no data leakage)- Backtest: 2022, 2023, 2024 independently- Hold-out test: 2025**Results**: See performance metrics above**Next Steps**:1. Hyperparameter tuning2. Additional features3. Learning-to-Rank4. Industry-level analysis